# Meal Predict Base Line - RandomForest
## score: 111.181

In [31]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [32]:
train_df = pd.read_csv("./data/origin_train.csv")
test_df = pd.read_csv("./data/origin_test.csv")
submission = pd.read_csv("./data/sample_submission.csv")

print(train_df.shape, test_df.shape)
train_df.head()

(1205, 12) (50, 10)


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


## columns name 변경

In [33]:
cols = ["date", "week_day", "member", "day_off", "business_trip", "overtime_accept", "work_from_home", 
        "breakfast", "lunch", "dinner", "total_lunch", "total_dinner"]

train_df.columns = cols
test_df.columns = cols[:-2]

train_df.columns.values, test_df.columns.values

(array(['date', 'week_day', 'member', 'day_off', 'business_trip',
        'overtime_accept', 'work_from_home', 'breakfast', 'lunch',
        'dinner', 'total_lunch', 'total_dinner'], dtype=object),
 array(['date', 'week_day', 'member', 'day_off', 'business_trip',
        'overtime_accept', 'work_from_home', 'breakfast', 'lunch',
        'dinner'], dtype=object))

In [34]:
# train, test update
train_df.to_csv("./data/train.csv", index=False)
test_df.to_csv("./data/test.csv", index=False)

## date → datetime 으로 type 변경

In [35]:
train_df["date"] = pd.to_datetime(train_df["date"], format="%Y-%m-%d")
test_df["date"] = pd.to_datetime(test_df["date"], format="%Y-%m-%d")

train_df["date"].head()

0   2016-02-01
1   2016-02-02
2   2016-02-03
3   2016-02-04
4   2016-02-05
Name: date, dtype: datetime64[ns]

In [36]:
test_df["date"].head()

0   2021-01-27
1   2021-01-28
2   2021-01-29
3   2021-02-01
4   2021-02-02
Name: date, dtype: datetime64[ns]

In [37]:
# train date rane
train_df["date"].min(), train_df["date"].max() 

(Timestamp('2016-02-01 00:00:00'), Timestamp('2021-01-26 00:00:00'))

In [38]:
# test date range
test_df["date"].min(), test_df["date"].max() 

(Timestamp('2021-01-27 00:00:00'), Timestamp('2021-04-09 00:00:00'))

## day numeric mapping

In [39]:
day = {'월':2, '화':3, '수':4, '목':5, '금':6}

train_df["week_day"] = train_df["week_day"].map(day)
test_df["week_day"] = test_df["week_day"].map(day)

train_df["week_day"].head()

0    2
1    3
2    4
3    5
4    6
Name: week_day, dtype: int64

## Select numeric features

In [40]:
x_train = train_df[['week_day', 'member', 'day_off', 'business_trip', 'overtime_accept', 'work_from_home']]
y_l = train_df['total_lunch']
y_d = train_df['total_dinner']

x_test = test_df[['week_day', 'member', 'day_off', 'business_trip', 'overtime_accept', 'work_from_home']]

## RandomForest Model

In [41]:
rf_l = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_l.fit(x_train, y_l)

rf_d = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_d.fit(x_train, y_d)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [42]:
y_l_pred = rf_l.predict(x_test)
y_d_pred = rf_d.predict(x_test)

In [43]:
submission['중식계'] = y_l_pred
submission['석식계'] = y_d_pred

submission.to_csv("./data/submission/baseline.csv", index=False)

## check submission

In [44]:
tmp = pd.read_csv("./data/submission/baseline.csv")
tmp.head()

,일자,중식계,석식계
0,2021-01-27,1057.51,387.09
1,2021-01-28,990.81,382.70
2,2021-01-29,583.58,255.98
3,2021-02-01,1204.97,450.56
4,2021-02-02,1012.39,404.72
